**WARNING: This is a rough workpad. Steps may not be sequential and there is no coherent narrative.**

In [1]:
import pandas as pd
import numpy as np
import nltk
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from csv import DictReader
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

import re


In [2]:
train = pd.read_csv('./../data/train.csv')
test = pd.read_csv('./../data/test.csv')

In [3]:
test.category.to_csv('./../pickles/test_targets.csv', header=True, index=False)

In [4]:
test.sample()

,category,text
1156,work,"It's not that I don't love my job, I just hate..."


In [5]:
#add msg length
train['msg_len'] = train['text'].apply(lambda msg: len(str(msg).decode('utf8').split(' ')))

In [6]:
#add message sentiment and polarity scores
train['polarity']=train.text.apply(lambda msg: TextBlob(str(msg).decode('utf-8')).sentiment[0])
train['subjectivity']=train.text.apply(lambda msg: TextBlob(str(msg).decode('utf-8')).sentiment[1])

In [7]:
train.shape

(14048, 5)

In [8]:
train.sample()

,category,text,msg_len,polarity,subjectivity
319,food,That awkward moment you have a hot waiter and ...,23,-0.175,0.925


In [9]:
#POS features
parts={
    'CC':0,
    'CD':0,
    'DT':0,
    'EX':0,
    'FW':0,
    'IN':0,
    'JJ':0,
    'JJR':0,
    'JJS':0,
    'LS':0,
    'MD':0,
    'NN':0,
    'NNS':0,
    'NNP':0,
    'NNPS':0,
    'PDT':0,
    'POS':0,
    'PRP':0,
    'PRP$':0,
    'RB':0,
    'RBR':0,
    'RBS':0,
    'RP':0,
    'SYM':0,
    'TO':0,
    'UH':0,
    'VB':0,
    'VBD':0,
    'VBG':0,
    'VBN':0,
    'VBP':0,
    'VBZ':0,
    'WDT':0,
    'WP':0,
    'WP$':0,
    'WRB':0}




for key in parts.keys():
    train[key]=0


for i in np.arange(0,train.shape[0]-1):
    for word,pos in TextBlob(str(train.text[i]).decode('utf-8')).tags:
        train.loc[i,pos] += 1

In [10]:
train.sample()

,category,text,msg_len,polarity,subjectivity,WRB,PRP$,VBG,FW,CC,...,JJR,SYM,VB,TO,UH,LS,RB,WDT,NNS,NNP
6493,school,Prom without a date is gona suck this year,9,0.0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
#now, generate document term matrix of EMOJIS

def emoji_parser(dat):
    """
    Create a emoticon regex vocabulary and return counts of each as features to the passed in dataframe
    """
#     emo_dict={'smile':re.compile(r':[-\)]')',
#     'wink':re.compile(r';[-\)p]'),
#     'sad':re.compile(r':[-\(]'),
#     'cool':re.compile(r'B\)|:>'),
#     'laugh':re.compile(r':[-dD]'),
#     'cry':re.compile(r'D:|:[-s]'),
#     'silly':re.compile(r':[-p]'),
#     'other':re.compile(r':[-ox\|\/\[@]|>:-\)'),
#     'pos_emos':re.compile(r':[\*]|o:\)|:3|\(y\)|<3'),
#     'neg_emos':re.compile(r':[-!]|>:-o|\(n\)')}
    
    emo_dict={'smile':':[-\)]',
    'wink':';[-\)p]',
    'sad':':[-\(]',
    'cool':'B\)|:>',
    'laugh':':[-dD]',
    'cry':'D:|:[-s]',
    'silly':':[-p]',
    'other':':[-ox\|\/\[@]|>:-\)',
    'pos_emos':':[\*]|o:\)|:3|\(y\)|<3',
    'neg_emos':':[-!]|>:-o|\(n\)'}
    
    #initialize emo columns
    for key in emo_dict.keys():
        dat[key] = 0
    
    #populate counts. For every message, check for key match and update count
    for i in np.arange(0, dat.shape[0]-1):
        msg = str(dat['text'][i]).decode('utf-8')
        
        for k, v in emo_dict.iteritems():
            pattern = re.compile(v)
            num_matches = len(pattern.findall(msg))
            if num_matches:
                dat.loc[i, k] += num_matches
    
    return dat

In [12]:
train = emoji_parser(train)

In [13]:
train[np.sum(train[['wink','cry','silly','laugh','pos_emos','smile','cool','other','neg_emos','sad']], axis=1)>0].sample(10)

,category,text,msg_len,polarity,subjectivity,WRB,PRP$,VBG,FW,CC,...,wink,cry,silly,laugh,pos_emos,smile,cool,other,neg_emos,sad
11409,relationships,I got together with someone and it was a long ...,40,-0.316667,0.641667,0,0,1,0,3,...,0,0,0,0,0,0,0,1,0,0
7888,misc,I couldn't sleep :-/,4,-0.250000,1.000000,0,0,0,0,0,...,0,1,1,1,0,1,0,1,1,1
5299,personal,This generation is not depressed 'because of s...,37,-0.098333,0.728333,0,2,3,0,1,...,0,0,0,0,0,0,0,0,0,1
9734,misc,Off to NZ :),4,0.500000,1.000000,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4666,personal,Wishing I could buy beer :(,6,-0.750000,1.000000,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
13089,relationships,I thought she might have been extraordinary. T...,15,-0.305556,0.966667,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4627,relationships,I'm in love with this guy but he is always tal...,24,0.156250,0.568750,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,1
13990,personal,I'm so over being upset all the time :(,9,-0.750000,1.000000,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
12396,misc,Fuuuuck I done did it again :-\,7,0.000000,0.000000,0,0,0,0,0,...,0,1,1,1,0,1,0,1,1,1
11793,personal,Bored sad and lonely nothing new:(,6,-0.342727,0.830909,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1


In [14]:
# Extract the is_hookup indicator flag (see Messages_Exploratory.ipynb)
train['is_hookup']=train.text.str.count(r'\s*\d{2,2}\s*[MmFf]|\s*[MmFf]\s*\d{1,2}|\d\d-\d\d')

In [15]:
train[train['is_hookup']>0].sample(5)

,category,text,msg_len,polarity,subjectivity,WRB,PRP$,VBG,FW,CC,...,cry,silly,laugh,pos_emos,smile,cool,other,neg_emos,sad,is_hookup
12341,personal,I just spent 30 minutes getting ready to send ...,13,0.050000,0.300000,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1.0
470,misc,IM SO BORED.............. 19M,4,-0.500000,1.000000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
8394,qna,"15m but I hangout with ages 17-25 mostly, me a...",22,0.044444,0.611111,0,1,0,0,2,...,0,0,0,0,0,0,0,0,0,2.0
6127,meetup,Any girls in SW burbs? 33m,6,0.000000,0.000000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
11983,meetup,Any gay guys 15-20 wanna chat,6,0.416667,0.583333,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0


In [ ]:
# now, need to tokenize the main text messages and vectorize

## lowercase
## remove stopwords
## remove punctuations and apostrophes
## consider len > 1
## remove numerics
## tfidf across entire document


In [16]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [17]:
def clean_msgs(msg):
    """
    Takes a string and returns an alphabet only, lowercase, stopped, stemmed, and tokenize-able string
    """
    
    #if missing, impute with "msngval"
    if not msg:
        msg=u'msngval'
    else:
        msg=str(msg).decode('UTF-8')
    
        #alphabets only
        msg = re.sub('[^a-zA-Z]',' ', msg)

        msg = msg.lower().split()

        #remove stopwords
        stops = set(stopwords.words('english'))
        msg = [word for word in msg if not word in stops]

        #stem
        p=PorterStemmer()
        msg = [p.stem(word) for word in msg]
    
    return " ".join(msg).decode('utf8')

In [18]:
corpus =[]
for i in np.arange(0,train.shape[0]):
    corpus.append(clean_msgs(train['text'][i]))

In [19]:
len(corpus)

14048

In [20]:
# Ready for BOW
tfidf_vect = TfidfVectorizer(lowercase=False,analyzer='word', ngram_range=(1,1), stop_words='english', 
                              use_idf=True, smooth_idf=True, min_df=1)

In [21]:
X = tfidf_vect.fit_transform(corpus)

In [22]:
X.toarray()[900].max()

0.68253678962209852

In [23]:
BOW_feats=X.toarray()

In [24]:
#impute missing values to 0 (NaNs resulted from empty strings)
train.iloc[:,2:]=train.iloc[:,2:].apply(lambda val: val.fillna(0), axis=1)

In [25]:
finX = np.concatenate((BOW_feats, train.iloc[:,2:]), axis=1)

In [30]:
finX.shape

(14048, 7598)

In [60]:
#xtrain, xtest, ytrain, ytest = train_test_split(finX, train.category, stratify=train.category)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [34]:
#test read
import h5py
with h5py.File('./../pickles/final_training_data.h5','r') as hf:
    print('List of arrays in this file: \n', hf.keys())
    data = hf.get('dataset_1')
    train_dat = np.array(data)
    print('Shape of the array dataset_1: \n', train_dat.shape)

('List of arrays in this file: \n', [u'dataset_1'])
('Shape of the array dataset_1: \n', (14048, 50))


In [36]:
xtrain, xtest, ytrain, ytest = train_test_split(train_dat, train.category, stratify=train.category)

In [39]:
clf_LR = LogisticRegression(class_weight=None)
clf_LR.fit(xtrain, ytrain)
preds = clf_LR.predict(xtest)

In [40]:
f1_score(ytest, preds, average='macro')

/Users/ash/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.13543211510514758

In [70]:
xtrain[:,7548:].shape

(10536, 50)

In [74]:
#scale test and train here
std_scale_train = preprocessing.MinMaxScaler().fit_transform(xtrain[:,7548:])
std_scale_test = preprocessing.MinMaxScaler().fit_transform(xtest[:,7548:])


Xtrain = np.concatenate((xtrain[:,:7548],xtrain[:,7548:]), axis=1)
Xtest = np.concatenate((xtest[:,:7548],xtest[:,7548:]), axis=1)

In [77]:
Xtrain.shape, Xtest.shape

((10536, 7598), (3513, 7598))

In [88]:
clf = LogisticRegression(class_weight='balanced')

In [89]:
clf.fit(Xtrain, ytrain)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [90]:
preds = clf.predict(Xtest)

In [91]:
macrof1 = f1_score(ytest, preds, average='macro')

In [92]:
macrof1

0.51201018622816563

In [84]:
from sklearn.metrics import classification_report


In [93]:
print (classification_report(ytest, preds))

             precision    recall  f1-score   support

    animals       0.67      0.76      0.71        37
      faith       0.23      0.25      0.24        12
     family       0.50      0.78      0.61       111
    fashion       0.24      0.50      0.33        34
       food       0.48      0.57      0.52        67
      lgbtq       0.75      0.60      0.66       136
     meetup       0.78      0.77      0.78       736
   military       0.38      0.45      0.42        11
       misc       0.49      0.47      0.48       680
   personal       0.62      0.50      0.55       890
pop_culture       0.38      0.44      0.40       130
        qna       0.33      0.36      0.34       227
relationships       0.55      0.57      0.56       290
     school       0.58      0.64      0.60        66
     sports       0.27      0.47      0.34        15
     tatoos       0.65      0.71      0.68        21
       work       0.36      0.70      0.47        50

avg / total       0.58      0.57      0.57

In [95]:
clf_rf = RandomForestClassifier(class_weight='balanced_subsample', n_jobs=-1, n_estimators=500)
clf_rf.fit(Xtrain, ytrain)

RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [99]:
rfpreds = clf_rf.predict(Xtest)
print (classification_report(ytest, rfpreds))
print ("MACRO: ", f1_score(ytest, rfpreds, average='macro'))


             precision    recall  f1-score   support

    animals       0.71      0.68      0.69        37
      faith       0.00      0.00      0.00        12
     family       0.69      0.30      0.42       111
    fashion       1.00      0.03      0.06        34
       food       0.67      0.15      0.24        67
      lgbtq       0.79      0.38      0.51       136
     meetup       0.67      0.80      0.73       736
   military       1.00      0.09      0.17        11
       misc       0.50      0.43      0.46       680
   personal       0.43      0.81      0.56       890
pop_culture       0.82      0.07      0.13       130
        qna       0.50      0.09      0.16       227
relationships       0.82      0.23      0.36       290
     school       0.73      0.24      0.36        66
     sports       0.00      0.00      0.00        15
     tatoos       0.76      0.62      0.68        21
       work       0.62      0.20      0.30        50

avg / total       0.59      0.53      0.49

In [100]:
#SVM
clf_svc = SVC(class_weight='balanced', kernel='linear')
clf_svc.fit(Xtrain, ytrain)
svmpreds = clf_svc.predict(Xtest)
print (classification_report(ytest, svmpreds))
print ("MACRO: ", f1_score(ytest, svmpreds, average='macro'))


             precision    recall  f1-score   support

    animals       0.00      0.00      0.00        37
      faith       0.00      0.00      0.00        12
     family       0.13      0.35      0.19       111
    fashion       0.03      0.15      0.05        34
       food       0.00      0.00      0.00        67
      lgbtq       0.00      0.00      0.00       136
     meetup       0.29      0.83      0.43       736
   military       0.01      0.09      0.01        11
       misc       0.14      0.00      0.00       680
   personal       0.49      0.08      0.14       890
pop_culture       0.16      0.10      0.12       130
        qna       0.14      0.08      0.10       227
relationships       0.22      0.21      0.22       290
     school       0.00      0.00      0.00        66
     sports       0.00      0.00      0.00        15
     tatoos       0.00      0.00      0.00        21
       work       0.05      0.06      0.05        50

avg / total       0.25      0.23      0.16

In [103]:
#GAUSSIAN NB

from sklearn.naive_bayes import GaussianNB



In [104]:
clf_nb = GaussianNB()
clf_nb.fit(Xtrain, ytrain)
nbpreds = clf_nb.predict(Xtest)
print (classification_report(ytest, nbpreds))
print ("MACRO: ", f1_score(ytest, nbpreds, average='macro'))


             precision    recall  f1-score   support

    animals       0.06      0.19      0.10        37
      faith       0.00      0.00      0.00        12
     family       0.12      0.26      0.16       111
    fashion       0.05      0.12      0.07        34
       food       0.16      0.28      0.21        67
      lgbtq       0.13      0.43      0.20       136
     meetup       0.43      0.17      0.25       736
   military       0.02      0.09      0.03        11
       misc       0.23      0.12      0.16       680
   personal       0.39      0.22      0.28       890
pop_culture       0.26      0.28      0.27       130
        qna       0.11      0.11      0.11       227
relationships       0.20      0.20      0.20       290
     school       0.10      0.21      0.13        66
     sports       0.10      0.20      0.14        15
     tatoos       0.02      0.24      0.04        21
       work       0.04      0.16      0.07        50

avg / total       0.28      0.19      0.21

In [105]:
from sklearn.decomposition import PCA


In [109]:
dimred = PCA(n_components=0.99)

In [110]:
red = dimred.fit_transform(Xtrain)

In [120]:
red.shape, Xtrain.shape

((10536, 30), (10536, 7598))

In [114]:
#30 features explain 99% of the variance in this model. This is awesome and helps us drop thousnds of features.
#Will it do as well as the original?
clf_pca = LogisticRegression(class_weight='balanced')
clf_pca.fit(red, ytrain)

preds_red = clf.predict(Xtest)
print (classification_report(ytest, preds_red))
print ("MACRO F1 = ", f1_score(ytest, preds_red, average='macro'))


             precision    recall  f1-score   support

    animals       0.67      0.76      0.71        37
      faith       0.23      0.25      0.24        12
     family       0.50      0.78      0.61       111
    fashion       0.24      0.50      0.33        34
       food       0.48      0.57      0.52        67
      lgbtq       0.75      0.60      0.66       136
     meetup       0.78      0.77      0.78       736
   military       0.38      0.45      0.42        11
       misc       0.49      0.47      0.48       680
   personal       0.62      0.50      0.55       890
pop_culture       0.38      0.44      0.40       130
        qna       0.33      0.36      0.34       227
relationships       0.55      0.57      0.56       290
     school       0.58      0.64      0.60        66
     sports       0.27      0.47      0.34        15
     tatoos       0.65      0.71      0.68        21
       work       0.36      0.70      0.47        50

avg / total       0.58      0.57      0.57

In [115]:
#NOT BAD AT ALL! PCA allowed us to use a much smaller model to get almost the same performance!

In [ ]:
BOW_feats=X.toarray()

In [ ]:
BOW_feats.shape

In [ ]:
tfidf_vect.get_feature_names()

In [ ]:
BOW_feats.shape

In [ ]:
train.columns

In [ ]:
#impute missing values to 0 (NaNs resulted from empty strings)
train.iloc[:,2:]=train.iloc[:,2:].apply(lambda val: val.fillna(0), axis=1)

In [ ]:
fin_X = np.concatenate((BOW_feats, train.iloc[:,2:]), axis=1)

In [ ]:
fin_X.shape

In [ ]:
from sklearn import preprocessing

In [ ]:
fin_X_scaled = preprocessing.scale(fin_X, axis=1)

In [ ]:
fin_X_scaled.shape

In [117]:
import h5py

In [ ]:
with h5py.File('./../pickles/train_feature_mat.h5', 'w') as hf:
    hf.create_dataset('dataset_1', data=fin_X_scaled, compression='gzip', compression_opts=9)

In [118]:
#test read
with h5py.File('./../pickles/train.h5','r') as hf:
    print('List of arrays in this file: \n', hf.keys())
    data = hf.get('dataset_1')
    np_data = np.array(data)
    print('Shape of the array dataset_1: \n', np_data.shape)

('List of arrays in this file: \n', [u'dataset_1'])
('Shape of the array dataset_1: \n', (14048, 7728))


In [119]:
np_data.shape

(14048, 7728)

In [ ]:
np_data

In [ ]:

train.category.to_csv('./../pickles/training_targets.csv',header=True, index=False)

In [ ]:
type(target.category)